In [75]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [76]:
from crewai import Task,Agent,Crew,LLM

In [77]:
!pip install dotenv
from dotenv import load_dotenv
import os
load_dotenv()
# Access your environment variables
api_key = os.getenv("GEMINI_API_KEY")

# Initialize LLM and tools
llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0,
    api_key=api_key,
)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [78]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm,
)

In [79]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [80]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [81]:
planner

Agent(role=Content Planner, goal=Plan engaging and factually accurate content on {topic}, backstory=You're working on planning a blog article about the topic: {topic}.You collect information that helps the audience learn something and make informed decisions. Your work is the basis for the Content Writer to write an article on this topic.)

In [82]:
editor

Agent(role=Editor, goal=Edit a given blog post to align with the writing style of the organization. , backstory=You are an editor who receives a blog post from the Content Writer. Your goal is to review the blog post to ensure that it follows journalistic best practices,provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics or opinions when possible.)

In [83]:
writer

Agent(role=Content Writer, goal=Write insightful and factually accurate opinion piece about the topic: {topic}, backstory=You're working on a writing a new opinion piece about the topic: {topic}. You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic. You follow the main objectives and direction of the outline, as provide by the Content Planner. You also provide objective and impartial insights and back them up with information provide by the Content Planner. You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.)

In [84]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [85]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    context=[plan]
)

In [86]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    context=[write]
)

In [87]:
plan

Task(description=1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources., expected_output=A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.)

In [88]:
write

Task(description=1. Use the content plan to craft a compelling blog post on {topic}.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.
, expected_output=A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.)

In [89]:
edit

Task(description=Proofread the given blog post for grammatical errors and alignment with the brand's voice., expected_output=A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.)

In [90]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True,
    manager_llm=llm
)

In [91]:
crew

Crew(id=97b3d25a-4356-45c0-b35b-b41a31255410, process=sequential, number_of_agents=3, number_of_tasks=3)

In [92]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 97b3d25a-4356-45c0-b35b-b41a31255410                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Content Plan: Understanding and Navigating the World of Artificial Intelligence**                            │
│                                                                                                                 │
│  **1. Prioritized Trends, Key Players, and Noteworthy News:**                                                   │
│                                                                                                                 │
│  * **Trends:** Generative AI (e.g., DALL-E 2, Midjourney, Stable Diffusion, ChatGPT), AI-powered drug           │
│  discovery, responsible AI development (explainable AI, bias mitigation), edge AI (AI processing at the device  │
│  level), quantum AI advancements.                                                                               │
│  * **Key Players:** OpenAI, Google (DeepMind), Microsoft, Amazon (AWS), Meta, Nvidia, various startups          │
│  specializing in specific AI applications (e.g., healthcare, finance).                                          │
│  * **Noteworthy News:** Recent breakthroughs in large language models, advancements in AI-driven automation     │
│  impacting various industries, ethical debates surrounding AI bias and job displacement, government             │
│  regulations and initiatives related to AI development and deployment.  (Specific examples of news articles     │
│  will be added during the writing phase, ensuring currency).                                                    │
│                                                                                                                 │
│                                                                                                                 │
│  **2. Target Audience Analysis:**                                                                               │
│                                                                                                                 │
│  * **Primary Audience:** Tech-savvy individuals, business professionals (especially in tech, healthcare,        │
│  finance), students and researchers interested in AI, individuals concerned about the societal impact of AI.    │
│  * **Secondary Audience:** General public interested in learning about AI, policymakers and regulators.         │
│  * **Interests:**  Understanding the capabilities and limitations of AI, the ethical implications of AI, the    │
│  potential impact of AI on various industries and society, learning about specific AI applications, career      │
│  opportunities in AI.                                                                                           │
│  * **Pain Points:**  Lack of understanding of AI concepts, fear of job displacement due to automation,          │
│  concerns about AI bias and misuse, difficulty in evaluating the credibility of AI-related information.         │
│                                                                                                                 │
│                                                                                                                 │
│  **3. Detailed Content Outline:**                                                                               │
│                                                                                                                 │
│  **I. Introduction (approx. 150 words)**               

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 80d01256-5855-4b6c-a77e-09335d963d47                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4b31e4cd-0ce9-4cd1-86e9-a6d112346f20                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Understanding and Navigating the World of Artificial Intelligence                                            │
│                                                                                                                 │
│  Artificial intelligence (AI) is rapidly transforming our daily lives. From personalized recommendations on     │
│  streaming services to voice assistants on smartphones, AI's influence is undeniable. This article explores     │
│  current trends, key players, and ethical considerations surrounding this transformative technology, aiming to  │
│  provide a balanced understanding of its potential and challenges.  The impact of AI is multifaceted and        │
│  requires careful consideration to maximize benefits while mitigating risks.                                    │
│                                                                                                                 │
│                                                                                                                 │
│  ## Key Trends Shaping the AI Revolution                                                                        │
│                                                                                                                 │
│  The AI field is evolving at an unprecedented rate, with several key trends shaping its trajectory. Generative  │
│  AI, with tools like DALL-E 2 and ChatGPT, has captured public attention due to its ability to generate         │
│  realistic images and text.  While offering exciting creative possibilities, concerns regarding misinformation  │
│  and copyright remain. Responsible development and deployment are crucial to harnessing generative AI's power   │
│  without causing harm.                                                                                          │
│                                                                                                                 │
│  Another significant trend is AI's application in healthcare. AI-powered drug discovery and diagnostics are     │
│  improving treatment development and accuracy. Personalized medicine, tailored to individual genetic profiles,  │
│  is also gaining momentum.  The potential benefits are substantial, with reports suggesting AI could unlock     │
│  significant value in the healthcare industry.  However, careful consideration must be given to data privacy    │
│  and algorithmic bias in these applications.                                                                    │
│                                                                                                                 │
│  Further advancements include Explainable AI (XAI), which aims to increase transparency in AI decision-making,  │
│  and Edge AI, which processes data locally, enhancing privacy and speed.  The development of ethical            │
│  guidelines and regulations is crucial for responsible AI development, requiring collaboration between          │
│  researchers, developers, and policymakers.  Quantum AI, while still in its early stages, holds the potential   │
│  for revolutionary advancements in processing power.                                                            │
│                                                                                                                 │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 23818e69-0139-4973-959d-a06681e4ebcf                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 97b3d25a-4356-45c0-b35b-b41a31255410                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Understanding and Navigating the World of Artificial Intelligence                              │
│                                                                                                                 │
│  Artificial intelligence (AI) is rapidly transforming our daily lives. From personalized recommendations on     │
│  streaming services to voice assistants on smartphones, AI's influence is undeniable. This article explores     │
│  current trends, key players, and ethical considerations surrounding this transformative technology, aiming to  │
│  provide a balanced understanding of its potential and challenges.  The impact of AI is multifaceted and        │
│  requires careful consideration to maximize benefits while mitigating risks.                                    │
│                                                                                                                 │
│                                                                                                                 │
│  ## Key Trends Shaping the AI Revolution                                                                        │
│                                                                                                                 │
│  The AI field is evolving at an unprecedented rate, with several key trends shaping its trajectory. Generative  │
│  AI, with tools like DALL-E 2 and ChatGPT, has captured public attention due to its ability to generate         │
│  realistic images and text.  While offering exciting creative possibilities, concerns regarding misinformation  │
│  and copyright remain. Responsible development and deployment are crucial to harnessing generative AI's power   │
│  without causing harm.                                                                                          │
│                                                                                                                 │
│  Another significant trend is AI's application in healthcare. AI-powered drug discovery and diagnostics are     │
│  improving treatment development and accuracy. Personalized medicine, tailored to individual genetic profiles,  │
│  is also gaining momentum.  The potential benefits are substantial, with reports suggesting AI could unlock     │
│  significant value in the healthcare industry.  However, careful consideration must be given to data privacy    │
│  and algorithmic bias in these applications.                                                                    │
│                                                                                                                 │
│  Further advancements include Explainable AI (XAI), which aims to increase transparency in AI decision-making,  │
│  and Edge AI, which processes data locally, enhancing privacy and speed.  The development of ethical            │
│  guidelines and regulations is crucial for responsible AI development, requiring collaboration between          │
│  researchers, developers, and policymakers.  Quantum AI, while still in its early stages, holds the potential   │
│  for revolutionary advancements in processing power.                                                            │
│                                                       

In [95]:
from IPython.display import Markdown
Markdown(result.raw)

# Understanding and Navigating the World of Artificial Intelligence

Artificial intelligence (AI) is rapidly transforming our daily lives. From personalized recommendations on streaming services to voice assistants on smartphones, AI's influence is undeniable. This article explores current trends, key players, and ethical considerations surrounding this transformative technology, aiming to provide a balanced understanding of its potential and challenges.  The impact of AI is multifaceted and requires careful consideration to maximize benefits while mitigating risks.


## Key Trends Shaping the AI Revolution

The AI field is evolving at an unprecedented rate, with several key trends shaping its trajectory. Generative AI, with tools like DALL-E 2 and ChatGPT, has captured public attention due to its ability to generate realistic images and text.  While offering exciting creative possibilities, concerns regarding misinformation and copyright remain. Responsible development and deployment are crucial to harnessing generative AI's power without causing harm.

Another significant trend is AI's application in healthcare. AI-powered drug discovery and diagnostics are improving treatment development and accuracy. Personalized medicine, tailored to individual genetic profiles, is also gaining momentum.  The potential benefits are substantial, with reports suggesting AI could unlock significant value in the healthcare industry.  However, careful consideration must be given to data privacy and algorithmic bias in these applications.

Further advancements include Explainable AI (XAI), which aims to increase transparency in AI decision-making, and Edge AI, which processes data locally, enhancing privacy and speed.  The development of ethical guidelines and regulations is crucial for responsible AI development, requiring collaboration between researchers, developers, and policymakers.  Quantum AI, while still in its early stages, holds the potential for revolutionary advancements in processing power.


## The Titans and Innovators of the AI Landscape

Several key players are driving AI innovation.  Companies like OpenAI, Google (DeepMind), Microsoft, and Amazon are heavily invested in AI research and development, integrating AI into their products and services.  Nvidia's role in providing the computational power for advanced AI applications is also significant.  A vibrant ecosystem of startups further contributes to AI innovation across various sectors.  This dynamic landscape, characterized by both collaboration and competition, is essential for fostering innovation and a healthy AI industry.


## Ethical Considerations and Societal Impact: Navigating the Challenges

The rapid advancement of AI presents significant ethical challenges.  AI bias, often stemming from biased training data, can perpetuate societal inequalities.  Job displacement due to automation is a major concern, requiring proactive measures such as reskilling and upskilling initiatives.  Privacy concerns related to data collection and use are also paramount.  The potential for AI misuse, particularly in autonomous weapons systems, raises serious ethical questions. Addressing these challenges requires a multi-faceted approach, including ethical guidelines, regulations, and public education.

The future of work in the age of AI necessitates adaptability and a willingness to acquire new skills. While some jobs may be displaced, new opportunities will also emerge.  Government policies and educational initiatives will play a crucial role in supporting workers during this transition.  A proactive and collaborative approach is essential to ensure a just and equitable transition to an AI-powered future.


## Conclusion: Embracing the Future of AI Responsibly

Artificial intelligence is fundamentally reshaping our world.  Understanding its potential, limitations, and ethical implications is crucial for navigating this transformative era.  Responsible development, deployment, and regulation are essential to ensure that AI benefits all of humanity.  Open dialogue, collaboration, and a prioritization of ethical considerations are key to harnessing AI's power for a more equitable and prosperous future.

**Call to Action:**  Learn more about AI through reputable sources such as the MIT Technology Review and the AI Now Institute.  Engage in informed discussions about the future of AI.